In [ ]:
import os
import glob
import math

import pandas as pd

from django.apps import apps
from django.db.utils import DataError
from django.core.exceptions import FieldDoesNotExist

In [ ]:
all_models = [x for x in apps.all_models['vases'].values()]

In [ ]:
for current_class in all_models:
    model_name = current_class.__name__
    source_file = os.path.join(settings.BASE_DIR, 'odeeg','data', 'dumps', 'forimport', '{}.xlsx'.format(model_name))
    try:
        df_data = pd.read_excel(source_file)
        print(source_file)
    except FileNotFoundError:
        df_data = False
    if isinstance(df_data, pd.DataFrame):
        df_keys = df_data.keys()
        nr_cols = len(df_keys)
        for i, row in df_data.iterrows():
            temp_item, _ = current_class.objects.get_or_create(legacy_id=row[0])
            col_counter = 0
            while col_counter < nr_cols:
                cur_attr = df_keys[col_counter]
                try:
                    cur_attr_type = current_class._meta.get_field(df_keys[col_counter]).get_internal_type()
                except FieldDoesNotExist:
                    cur_attr_type = None
                if cur_attr_type is not None:
                    if "{}".format(cur_attr_type) == "CharField":
                        try:
                            my_val = "{}".format(row[cur_attr])[:249]
                            setattr(temp_item, cur_attr, my_val)
                        except TypeError:
                            pass
                    elif "{}".format(cur_attr_type) == "ForeignKey" and isinstance(row[cur_attr], str):
                        fk  = current_class._meta.get_field(df_keys[col_counter])
                        temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=row[cur_attr])
                        setattr(temp_item, cur_attr, temp_rel_obj)
                    elif "{}".format(cur_attr_type) == "ManyToManyField" and isinstance(row[cur_attr], str):
                        fk = current_class._meta.get_field(df_keys[col_counter])
                        rel_things = []
                        for x in row[cur_attr].split(";"):
                            temp_rel_obj, _ = fk.related_model.objects.get_or_create(legacy_id=x.strip())
                            rel_things.append(temp_rel_obj)
                            m2m_attr = getattr(temp_item, cur_attr)
                        m2m_attr.set(rel_things)
                    elif "{}".format(cur_attr_type) == "TextField" and isinstance(row[cur_attr], str):
                        setattr(temp_item, cur_attr, row[cur_attr])
                    elif "{}".format(cur_attr_type) == "DecimalField":
                        try:
                            value = float(row[cur_attr])
                        except ValueError:
                            
                            print("{} can't be parsed as float".format(row[cur_attr]))
                            value = math.nan
                        if value is not None:
                            setattr(temp_item, cur_attr, value)
                temp_item.save()
                col_counter += 1
#                 print(col_counter)

In [ ]:
# for x in all_models:
#     [y.delete() for y in x.objects.all()]